> This notebook is still a work-in-progress and has NOT been implemented yet!

In [ ]:
import netCDF4 as nc
import os
import datetime

In [27]:
original_file_path = "/home/prateiksinha/new_data/mpi/tas/tas_6hrPlevPt_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_199001010600-199501010000.nc"
original_dataset = nc.Dataset(original_file_path, "r")

second_file_path = "/home/prateiksinha/new_data/mpi/tas/tas_6hrPlevPt_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_199501010600-200001010000.nc"
second_dataset = nc.Dataset(second_file_path, "r")

In [ ]:
import os
import netCDF4 as nc
import numpy as np
from datetime import datetime, timedelta


# Path to your original .nc file
input_nc_file = "/home/prateiksinha/new_data/mpi/tas/tas_6hrPlevPt_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_199501010600-200001010000.nc"

# Directory to store the individual yearly .nc files
output_dir = "/home/prateiksinha/new_data/processed/mpi"

# Open the original .nc file for reading
with nc.Dataset(input_nc_file, "r") as ds:
    # Assuming the time variable is named "time" and has units in days since a reference date
    time_var = ds.variables["time"]
    
    # Convert time units to days since the reference date
    reference_date = datetime(1850, 1, 1, 0, 0, 0)
    time_days = time_var[:]
    
    print("TIME_DAYS", time_days)

    time_years = np.array([(reference_date + timedelta(days=float(days))).year for days in time_days])
    print("TIME_YEARS", time_years)
    
    # Loop through each year
    for year in range(int(time_years[0]), int(time_years[-1]) + 1):
        print("GOT THE YEAR", year)

        # Create a new .nc file for the current year
        output_nc_file = os.path.join(output_dir, f"climate_data_{year}.nc")
        with nc.Dataset(output_nc_file, "w") as output_ds:
            # Copy global attributes from the original file
            for attr_name in ds.ncattrs():
                setattr(output_ds, attr_name, getattr(ds, attr_name))
            
            # Create dimensions in the output file
            output_ds.createDimension("time", sum(year == time_years))  # Each year has one time step
            output_ds.createDimension("lat", len(ds.dimensions["lat"]))
            output_ds.createDimension("lon", len(ds.dimensions["lon"]))
            
            # Copy variables to the output file
            for var_name in ds.variables:
                if 'bnds' not in var_name:
                    var = ds.variables[var_name]
                    print("CURR VAR NAME", var_name)
                    print("ASSOCIATED VAR", dir(var))
                    output_var = output_ds.createVariable(var_name, var.dtype, var.dimensions)
                    
                    if "bnds" in var.dimensions:
                        # Handle bounds dimensions, e.g., for time variable
                        output_var[:] = var[:, year*2:year*2+2, :, :]
                    elif var_name == "time":
                        output_time_var = output_var
                        output_time_var.units = time_var.units
                        output_time_var.calendar = time_var.calendar
                        output_time_var[:] = time_days[time_years == year]  
                    elif var_name == 'tas':
                        output_var[:] = var[time_years == year]
                    else:
                        output_var[:] = var[:]
    
        print(f"Processed year {year}")